**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
sudo docker run --rm -v <ruta a mi directorio>:/datos --cap-add SYS_ADMIN --privileged --name jupyterlab-numerical -p 8888:8888 -d palmoreck/jupyterlab_numerical:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_numerical
```


Documentación de la imagen de docker `palmoreck/jupyterlab_numerical:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/numerical).

**Para ejecutar el comando de linux [perf](https://github.com/torvalds/linux/tree/master/tools/perf), que se revisará en esta nota, obsérvese que es necesario correr el contenedor de docker descrito antes con las *flags* `--cap-add SYS_ADMIN` y `--privileged`.**

---

Nota generada a partir de [liga](https://www.dropbox.com/s/fyqwiqasqaa3wlt/3.1.1.Multiplicacion_de_matrices_y_estructura_de_datos.pdf?dl=0), [liga2](https://www.dropbox.com/s/l4hq45rj860ql9f/3.1.2.Localidad_y_vectorizacion.Analisis_del_error_en_computos_matriciales_basicos.pdf?dl=0)

# 3.1 El cómputo matricial y el álgebra lineal. Vectorización, BLAS y el uso del caché eficientemente.

El cómputo matricial está construído sobre una jerarquía de operaciones del álgebra lineal:

* Productos punto involucran operaciones escalares de suma y multiplicación (nivel BLAS 1).

* La multiplicación matriz-vector está hecha de productos punto (nivel BLAS 2).

* La multiplicación matriz-matriz utiliza colecciones de productos matriz-vector (nivel BLAS 3).

Las operaciones anteriores se describen en el álgebra lineal con la teoría de espacios vectoriales pero también es posible describirlas en una forma algorítmica. Ambas descripciones se complementan una a la otra.

Manejaremos nombres que en el [Linear Algebra Package: LAPACK](http://www.netlib.org/lapack/explore-html/dir_fa94b7b114d387a7a8beb2e3e22bf78d.html) son utilizados para denotar operaciones con escalares, vectores o matrices. Ver [ Reference-LAPACK / lapack](https://github.com/Reference-LAPACK/lapack) para su github.

# Nivel 1 de BLAS

## Operación del producto interno estándar o producto punto

Consideramos $x,y \in \mathbb{R}^n$. El producto punto entre $x$ y $y$ es $c = x^Ty = \displaystyle \sum_{i=1}^n x_iy_i$. 

**Ejemplo y algoritmo del producto punto:**

In [1]:
c=0
n=5
x=[-1]*n
y=[1.5]*n

for i in range(n):
    c += x[i]*y[i]

In [2]:
c

-7.5

**Obs:**

* El producto punto de dos $n$-vectores involucran $n$ multiplicaciones y $n$ sumas para un total de $2n$ operaciones. Usamos la notación $\mathcal{O}(\cdot)$ para escribir que el producto punto es $\mathcal{O}(n)$ y se lee "de orden $n$ o proporcional a $n$" para indicar que la **cantidad de trabajo** tiene un comportamiento **lineal** con la dimensión $n$. También tal cantidad de trabajo opera sobre una **cantidad lineal de datos**.

* En LAPACK encontramos [sdot](http://www.netlib.org/lapack/explore-html/d0/d16/sdot_8f.html), [ddot](http://www.netlib.org/lapack/explore-html/d5/df6/ddot_8f.html), [cdotu](http://www.netlib.org/lapack/explore-html/d7/d7b/cdotu_8f.html) y [zdotu](http://www.netlib.org/lapack/explore-html/db/d2d/zdotu_8f.html) para descripción de las funciones/subrutinas escritas en [Fortran](https://en.wikipedia.org/wiki/Fortran) del producto punto en los casos de precisión simple, doble o números complejos respectivamente.

## Operación **saxpy**

Consideramos $\alpha \in \mathbb{R}, x,y \in \mathbb{R}^n$. El nombre lo recibe por *scalar alpha x plus y*. En LAPACK [saxpy](http://www.netlib.org/lapack/explore-html/d8/daf/saxpy_8f.html) se escribe en forma *update*:

$$y=\alpha x + y \therefore y_i = \alpha x_i + y_i \forall i=1,...,n$$

**Obs:** 

* El símbolo $=$ no se utiliza como igualdad de expresiones sino para denotar asignación (como en computación al escribir un algoritmo).

* También encontramos en LAPACK [caxpy](http://www.netlib.org/lapack/explore-html/de/da2/caxpy_8f.html) o [daxpy](http://www.netlib.org/lapack/explore-html/d9/dcd/daxpy_8f.html) para el caso complejo y para números en doble precisión respectivamente.

* Ésta operación realiza un trabajo de $\mathcal{O}(n)$ sobre una cantidad de datos $\mathcal{O}(n)$.

**Ejemplo y algoritmo de saxpy:**

In [3]:
alpha=2
n=5
x=[-2]*n
y=[0]*n

for i in range(n):
    y[i] += alpha*x[i]

In [4]:
y

[-4, -4, -4, -4, -4]

o en una forma *update*:

In [5]:
alpha=2
n=5
x=[-2]*n
y=[3,4,-1,0,1]

for i in range(n):
    y[i] += alpha*x[i]

In [6]:
y

[-1, 0, -5, -4, -3]

**Comentario:** La operación de producto punto y *saxpy* son algoritmos catalogados como de **nivel BLAS 1**, ver [BLAS: Basic Linear Algebra Subprograms](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms). Éstos algoritmos se caracterizan por involucrar una cantidad de trabajo lineal sobre una cantidad lineal de datos. Ver [level 1](http://www.netlib.org/blas/#_level_1) para más ejemplos de este tipo de algoritmos.

# Nivel 2 de BLAS

## Multiplicación matriz-vector

Consideramos $A \in \mathbb{R}^{m \times n}, x \in \mathbb{R}^n, y \in \mathbb{R}^m$. La operación $y = y + Ax$ es una operación *generalizada* saxpy, por ello se denomina **gaxpy** pero en LAPACK podemos encontrarla con nombres como [sgemv](http://www.netlib.org/lapack/explore-html/db/d58/sgemv_8f.html), [dgemv](http://www.netlib.org/lapack/explore-html/dc/da8/dgemv_8f.html), [cgemv](http://www.netlib.org/lapack/explore-html/d4/d8a/cgemv_8f.html) o [zgemv](http://www.netlib.org/lapack/explore-html/db/d40/zgemv_8f.html) para los casos de precisión simple, doble o números complejos respectivamente. Hay diferentes formas de visualizar y escribir el algoritmo de multiplicación matriz-vector. Por ejemplo para una matriz $A$ con entradas:


In [7]:
m=2
n=5
A=[[1.2]*n if i%2==0 else [1]*n for i in range(m)]

In [8]:
A

[[1.2, 1.2, 1.2, 1.2, 1.2], [1, 1, 1, 1, 1]]

In [9]:
A[0][0]

1.2

In [10]:
A[1][n-1]

1

se tiene:

### -) Algoritmo gaxpy *row oriented*

**Ejemplo**

In [11]:
x=[2]*n
y=[0]*m
for i in range(m):
    for j in range(n):
        y[i]+=A[i][j]*x[j]


In [12]:
y

[12.0, 10]

Si $y$ tiene valores distintos de $0$, se realiza un *update*:

In [13]:
x=[2]*n
y=[-1]*m
for i in range(m):
    for j in range(n):
        y[i]+=A[i][j]*x[j]


In [14]:
y

[11.0, 9]

**Comentarios:**

* En la versión *row oriented* del algoritmo *gaxpy*, el **inner loop** realiza **productos punto** entre el $i$-ésimo renglón de $A$ y el vector $x$. Se realizan $m$ productos punto $A[i,:]^Tx$:

```
for i in range(m)
    y[i]+=A[i,:]*x #producto punto
    
```

donde: $A[i,:]$ es el $i$-ésimo renglón de $A$. Así podemos reescribir de forma más compacta este algoritmo.

* Obsérvese que el acceso a la matriz $A$ del algoritmo *gaxpy row oriented* es **por renglón**.

También puede escribirse al algoritmo *gaxpy* en una forma orientada por columnas:

### -) Algoritmo gaxpy *column oriented*

Este algoritmo ayuda a visualizar al producto matriz-vector como una combinación lineal de las columnas de $A$:

$$Ax = \displaystyle \sum_{j=1}^n A_jx_j$$

con $A_j$ la $j$-ésima columna de $A$.


**Ejemplo**

In [15]:
x=[2]*n
y=[0]*m
for j in range(n):
    for i in range(m):
        y[i]+=A[i][j]*x[j]

In [16]:
y

[12.0, 10]

**Comentarios:**

* El algoritmo de multiplicación matriz-vector (versión *row* o *column* oriented) involucra $\mathcal{O}(mn)$ operaciones o una cantidad **cuadrática** de trabajo, que podemos entender como "si duplicamos cada dimensión de $A$ entonces la cantidad de trabajo se incrementa por un factor de $4$". Tal número de operaciones trabajan sobre una matriz o sobre una cantidad **cuadrática** de datos. A los algoritmos que realizan una cantidad cuadrática de trabajo sobre una cantidad cuadrática de datos se les cataloga de **nivel BLAS 2**. Ver [level 2](http://www.netlib.org/blas/#_level_2) para más ejemplos de algoritmos en el álgebra lineal en esta categoría.

* En el algoritmo *gaxpy column oriented* el acceso a la matriz $A$ es por columna.

* La versión *column oriented* se puede analizar desde el punto de vista puramente algorítmico como un intercambio entre las líneas con los índices $i$ y $j$ de cada *loop* y un acceso a los datos de la matriz por columna. O bien, se puede analizar desde el álgebra lineal indicando que el vector $y$ está en el **espacio generado** por las columnas de $A$ y cuyas coordenadas son dadas por las entradas del vector $x$:

<img src="https://dl.dropboxusercontent.com/s/6a2b7rjs4a71sni/combinacion_lineal_columnas_A.png?dl=0" heigth="700" width="700">

Una ejemplo de visualización del espacio generado por las columnas de $A \in \mathbb{R}^{3 \times 2}$, llamado **rango o imagen** de $A$, $Im(A)$, es el siguiente:

<img src="https://dl.dropboxusercontent.com/s/zkbhzv9a2jiw11b/espacio_generado_columnas_de_A.png?dl=0" heigth="400" width="400">

En este dibujo los vectores $b, r(x) \in \mathbb{R}^3$ no están en $Im(A) \subset \mathbb{R}^3$ pero $Ax$ en azul sí. $Im(A)$ en el dibujo es un plano en el espacio $\mathbb{R}^3$.



* Obsérvese que el **inner loop** de la versión *column oriented* en *gaxpy* es un **saxpy** en la que el escalar está dado por una entrada de $x$. Esto lo podemos escribir de forma explícita definiendo $A[:,j]$ a la $j$-ésima columna de $A$ por lo que $A = [A[:,1] | A[:,2] | \dots | A[:,n]]$, entonces:

```
for j in range(n)
    y+=A[:,j] * x[j]
 
```

sin embargo como hemos visto en Python con su implementación más común CPython, no es posible realizar tal indexado:

In [17]:
x=[2]*n
y=[0]*m
for j in range(n):
    y+=A[:,j]*x[j]

TypeError: list indices must be integers or slices, not tuple

a menos que incorporemos alguna paquetería que permita la **vectorización** y el uso de índices para extracción de columnas (o renglones) de $A$. Entre las razones que encontramos en Python el no soporte para la vectorización está que el [bytecode](https://en.wikipedia.org/wiki/Bytecode) de Python no está optimizado para vectorización. Un ejemplo de un paquete que permite realizar operaciones de forma vectorizada es [numpy](https://numpy.org/):

In [18]:
import numpy as np

In [19]:
x = 2*np.ones(n)
y = np.zeros(m)

In [20]:
x

array([2., 2., 2., 2., 2.])

In [21]:
y

array([0., 0.])

In [23]:
A=np.array([[1.2,1.2,1.2,1.2,1.2],[1,1,1,1,1]])

In [24]:
A

array([[1.2, 1.2, 1.2, 1.2, 1.2],
       [1. , 1. , 1. , 1. , 1. ]])

In [25]:
for j in range(n):
    y+=A[:,j]*x[j]

In [26]:
y

array([12., 10.])

Asimismo, el algoritmo *gaxpy row oriented* puede escribirse de forma más compacta haciendo uso de la definición de producto punto estándar: $x^Ty$ para dos vectores columna $x$ y $y$. En el caso de una matriz $A$ se tiene:

```
for i=1:m
    y[i]+=A[i,:]^T*x
```

donde: $A[i,:]$ es el $i$-ésimo renglón de $A$. En Python:

In [76]:
x = 2*np.ones(n)
y = np.zeros(m)
A=np.array([[1.2,1.2,1.2,1.2,1.2],[1,1,1,1,1]])

In [81]:
for i in range(m):
    y[i]+=A[i,:].dot(x)

In [82]:
y

array([12., 10.])

en donde se utilizó la función [numpy.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html) de *numpy*.

**Comentarios:**

* La vectorización como se describe en [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb), es una herramienta que tenemos a nuestra disposición para escribir programas de alto rendimiento. La vectorización incrementa\* el número de instrucciones por ciclo [IPC](https://en.wikipedia.org/wiki/Instructions_per_cycle) para procesadores que soportan el Single Instruction Multiple Data (SIMD) que encontramos en la taxonomía de Flynn (ver [liga](https://en.wikipedia.org/wiki/Flynn%27s_taxonomy)). Como ejemplo de tales procesadores están los procesadores vectoriales o en arreglo, ver [liga](https://en.wikipedia.org/wiki/Vector_processor). Obsérvese en [liga](https://github.com/numpy/numpy/blob/master/numpy/core/src/umath/simd.inc.src) que `numpy` soporta tales operaciones.

\*El incremento en IPC vía la vectorización se enfoca a que las instrucciones tiendan a completar trabajo **útil** por ciclo pues podría darse la situación en la que se tiene: *a high rate of instructions, but a low rate of actual work completed*.


#### `Perf`

En Linux existe la herramienta [perf](https://github.com/torvalds/linux/tree/master/tools/perf) que nos ayuda a calcular métricas de desempeño de la CPU o de los cores. Para la ejecución de los siguientes ejemplos es **indispensable** correr el contenedor de docker descrito al inicio de la nota en un sistema ubuntu.

**Ejemplo de cálculo de la norma $2$ al cuadrado de un vector**

In [27]:
%%file norm_square.py
n=10**5
vector=list(range(n))
norm=0
for v in vector:
    norm+=v*v

Overwriting norm_square.py


In [28]:
%%bash
sudo perf stat -S -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square.py


 Performance counter stats for 'python3 norm_square.py' (20 runs):

         112969642      cycles                                                        ( +-  0.36% )
         217301216      instructions              #    1.92  insn per cycle           ( +-  0.73% )
           1137719      cache-references                                              ( +-  0.92% )
             58733      cache-misses              #    5.162 % of all cache refs      ( +-  2.07% )

       0.030025947 seconds time elapsed                                          ( +-  1.22% )



**Comentarios:**

* Con `perf` se repiten las mediciones utilizando la *flag* `-r`. Con la *flag* `-e` se enlistan las métricas a calcular. `-S` llama a sync() antes de iniciar una ejecución.

* En el ejemplo anterior además de los ciclos e instrucciones se calculan los *cache references* y los *cache misses*. Ver [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb) para definiciones de *cache references* y *misses*. Esencialmente el sistema de memoria buscó el número que aparece en *cache-references* por datos o instrucciones en el caché y de éstas búsquedas, en *cache-misses* se repora el número (y porcentaje) de búsquedas fallidas que no estuvieron en memoria. 

* La fragmentación, ver [Fragmentation](https://en.wikipedia.org/wiki/Fragmentation_(computing)), incrementa el número de transferencias de memoria hacia la CPU y además imposibilita la vectorización porque el caché no está lleno. El caché puede llenarse sólo al tener bloques contiguos de memoria alojados para los datos (recuérdese que la interconexión o *bus* sólo puede mover *chunks* de memoria contigua, ver [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb)). Python en su implementación común CPython, tiene **data fragmentation** por ejemplo al usar listas. Las listas de Python alojan locaciones donde se pueden encontrar los datos y no los datos en sí. Al utilizar tales estructuras para operaciones matriciales el *runtime* de Python debe realizar *lookups* por índices y al no encontrarse de forma contigua los datos, el *overhead* en la transferencia de datos es mayor lo que causa que se tengan mayor número de *cache-misses* y que los cores tengan que esperar hasta que los datos estén disponibles en el caché.

* Obsérvese que se reporta el IPC al lado de *instructions* con nombre *insn per cycle*. Un alto IPC indica una alta transferencia de instrucciones de la unidad de memoria hacia la unidad de cómputo y un menor IPC indica más **stall cycles**, ver [pipeline stall](https://en.wikipedia.org/wiki/Pipeline_stall). Sin embargo como se mencionó antes, se debe evaluar si *a high rate of instructions* indica un *high rate of actual work completed* (p.ej. un *loop* tiene una alta tasa de IPC pero no siempre se realiza trabajo útil). Ver sección *CPU statistics* en la [liga](http://www.brendangregg.com/perf.html).

También podemos obtener las estadísticas por core. Obsérvese que la salida anterior de `perf` calculó el IPC a partir del número de instrucciones y ciclos ahí reportados. El mismo cálculo se realiza a continuación:

In [29]:
%%bash
sudo perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square.py


 Performance counter stats for 'system wide' (20 runs):

S0-C0           2          111124270      cycles                                                      
S0-C0           2          214054021      instructions              #    1.93  insn per cycle         
S0-C0           2            1196096      cache-references                                            
S0-C0           2              60653      cache-misses              #    5.071 % of all cache refs    
S0-C1           2          111521501      cycles                                                      
S0-C1           2           26249177      instructions              #    0.24  insn per cycle         
S0-C1           2               2337      cache-references                                            
S0-C1           2               1379      cache-misses              #   59.007 % of all cache refs    
S0-C2           2            1184308      cycles                                                      
S0-C2          

Si ejecutamos `perf` sólo con la *flag* `-r` resulta:

In [30]:
%%bash
sudo perf stat -S -r 20 python3 norm_square.py


 Performance counter stats for 'python3 norm_square.py' (20 runs):

         30.059631      task-clock (msec)         #    0.994 CPUs utilized            ( +-  1.01% )
                 0      context-switches          #    0.008 K/sec                    ( +- 39.74% )
                 0      cpu-migrations            #    0.000 K/sec                  
              2076      page-faults               #    0.069 M/sec                    ( +-  0.02% )
         111057933      cycles                    #    3.695 GHz                      ( +-  0.23% )
         214464593      instructions              #    1.93  insn per cycle           ( +-  0.45% )
          45107448      branches                  # 1500.599 M/sec                    ( +-  0.52% )
            669749      branch-misses             #    1.48% of all branches          ( +-  0.11% )

       0.030241277 seconds time elapsed                                          ( +-  1.02% )



**Comentarios:**

* La métrica de *task-clock* indica cuántos *clock cycles* tomó nuestra tarea y se reporta en milisegundos. Obsérvese que también se indica cuántos CPU's fueron utilizados, no es exactamente 1 pues el programa no sólo involucra trabajo de CPU sino también de alojamiento de memoria. 

* *context-switches* y *cpu-migrations* indican cuánto se detuvo el programa para realizar:

    * operaciones relacionadas con el kernel del sistema (por ejemplo I/O).
    * ejecución de otras aplicaciones.
    * alojamiento de la ejecución en un core distinto (lo que ocasiona que haya movimiento de datos o instrucciones hacia el caché de otro core).
    
la idea es tener un número pequeño de éstas métricas.

* *page-faults* se relaciona con el alojamiento de memoria a partir del kernel del sistema operativo. Ocasiona que se detenga la ejecución del programa y por tanto deseamos que ésta métrica no sea muy grande. Ver [Page fault](https://en.wikipedia.org/wiki/Page_fault).

* Ver [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb) para el *branching*. Esencialmente interesa que el número de *branch misses* sea pequeño.



Podemos generar la salida anterior por *core*:

In [31]:
%%bash
sudo perf stat -S -a --per-core -r 20 python3 norm_square.py


 Performance counter stats for 'system wide' (20 runs):

S0-C0           2          59.699698      cpu-clock (msec)          #    1.911 CPUs utilized          
S0-C0           2                  5      context-switches          #    0.084 K/sec                  
S0-C0           2                  0      cpu-migrations            #    0.000 K/sec                  
S0-C0           2               2074      page-faults               #    0.035 M/sec                  
S0-C0           2          114151283      cycles                    #    1.912 GHz                    
S0-C0           2          226146639      instructions              #    1.98  insn per cycle         
S0-C0           2           47873603      branches                  #  801.907 M/sec                  
S0-C0           2             667536      branch-misses             #    1.39% of all branches        
S0-C1           2          59.700993      cpu-clock (msec)          #    1.911 CPUs utilized          
S0-C1          

Para contrastar las salidas anteriores con un mejor uso del caché se utiliza a continuación el paquete de `numpy`:

In [33]:
%%file norm_square_numpy.py
import numpy as np
n=10**5
vector=np.arange(n)
vector.dot(vector)

Writing norm_square_numpy.py


In [32]:
%%bash
sudo perf stat -S -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'python3 norm_square_numpy.py' (20 runs):

        2517939029      cycles                                                        ( +-  0.07% )
        1287233993      instructions              #    0.51  insn per cycle           ( +-  0.07% )
           7472851      cache-references                                              ( +-  0.22% )
            403479      cache-misses              #    5.399 % of all cache refs      ( +-  1.80% )

       0.139254850 seconds time elapsed                                          ( +-  0.36% )



**Comentario:** 

* Obsérvese que es **más tardado** este programa para el número $n$ que se está utilizando en comparación con el programa anterior sin vectorizar.

* El número de instrucciones es **menor** al reportado anteriormente y parece tener un mayor número $%$ de *cache-misses*. Obsérvese que para este ejemplo es más ilustrativo observar las estadísticas por *core* y realizar conclusiones:

In [33]:
%%bash
sudo perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'system wide' (20 runs):

S0-C0           2          569110059      cycles                                                      
S0-C0           2          222173905      instructions              #    0.39  insn per cycle         
S0-C0           2              47498      cache-references                                            
S0-C0           2              12696      cache-misses              #   26.730 % of all cache refs    
S0-C1           2          569620862      cycles                                                      
S0-C1           2          223486329      instructions              #    0.39  insn per cycle         
S0-C1           2              51512      cache-references                                            
S0-C1           2              12984      cache-misses              #   25.206 % of all cache refs    
S0-C2           2          569812750      cycles                                                      
S0-C2          

**Comentarios:** 

* Se observa que la métrica IPC está más uniforme a través de los cores lo que indica un mejor *load balancing*.

* Además, el $\%$ de caché misses es menor que en el caso no vectorizado.

También podemos obtener las salidas anteriores de `perf` al usar `numpy`: 

In [34]:
%%bash
sudo perf stat -S -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'python3 norm_square_numpy.py' (20 runs):

        658.846611      task-clock (msec)         #    4.733 CPUs utilized            ( +-  0.28% )
              1309      context-switches          #    0.002 M/sec                    ( +- 97.65% )
                 1      cpu-migrations            #    0.002 K/sec                    ( +- 14.60% )
              4755      page-faults               #    0.007 M/sec                    ( +-  0.04% )
        2509194053      cycles                    #    3.808 GHz                      ( +-  0.28% )
        1288129520      instructions              #    0.51  insn per cycle           ( +-  0.14% )
         256581259      branches                  #  389.440 M/sec                    ( +-  0.15% )
           6512464      branch-misses             #    2.54% of all branches          ( +-  0.09% )

       0.139189866 seconds time elapsed                                          ( +-  0.27% )



In [35]:
%%bash
sudo perf stat -S -a --per-core -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'system wide' (20 runs):

S0-C0           2         280.327176      cpu-clock (msec)          #    1.997 CPUs utilized          
S0-C0           2                 48      context-switches          #    0.171 K/sec                  
S0-C0           2                  5      cpu-migrations            #    0.018 K/sec                  
S0-C0           2                  4      page-faults               #    0.014 K/sec                  
S0-C0           2          569732162      cycles                    #    2.032 GHz                    
S0-C0           2          222998243      instructions              #    0.39  insn per cycle         
S0-C0           2           41820920      branches                  #  149.186 M/sec                  
S0-C0           2             761701      branch-misses             #    1.82% of all branches        
S0-C1           2         280.362219      cpu-clock (msec)          #    1.997 CPUs utilized          
S0-C1          

### Regresando al algoritmo *gaxpy column oriented*...

Utilizaremos `perf` en lo que sigue para evaluar las métricas de IPC, *cache-references*, *cache-misses* y medir el tiempo de ejecución. Además, esto permitirá comparar los tiempos de ejecución del algoritmo *gaxpy* (nivel BLAS 2), con vectorización y sin vectorización:

In [44]:
np.random.seed(2020)
m=10**4
n=10**4
A=np.random.rand(m,n)
file='A.txt'
np.savetxt(file,A)

Arhivo sin vectorizar y utilizando listas para la matriz y los vectores:

In [37]:
%%file mult_matrix_vector.py
m=10**4
n=10**4
x=[2]*n
y=[0]*m
A = []
file='A.txt'
with open(file,'r') as f:
    for l in f:
        A.append([float(k) for k in l.replace('\n','').replace(' ',',').split(',')])      
for j in range(n):
    for i in range(m):
        y[i]+=A[i][j]*x[j]

Writing mult_matrix_vector.py


Archivo vectorizando con numpy:

In [38]:
%%file mult_matrix_vector_numpy.py
import numpy as np
m=10**4
n=10**4
x = 2*np.ones(n)
y = np.zeros(m)
file='A.txt'
A = np.loadtxt(file)
for j in np.arange(n):
    y+=A[:,j]*x[j]

Writing mult_matrix_vector_numpy.py


#### Caso no vectorizado

In [39]:
%%bash
sudo perf stat -S -e cycles,instructions,cache-references,cache-misses -r 2 python3 mult_matrix_vector.py


 Performance counter stats for 'python3 mult_matrix_vector.py' (2 runs):

      259544977496      cycles                                                        ( +-  0.60% )
      625505659134      instructions              #    2.41  insn per cycle           ( +-  0.13% )
        1014372934      cache-references                                              ( +-  0.28% )
         126123343      cache-misses              #   12.434 % of all cache refs      ( +-  0.87% )

      65.051571959 seconds time elapsed                                          ( +-  0.63% )



Métricas por core:

In [40]:
%%bash
sudo perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 2 python3 mult_matrix_vector.py


 Performance counter stats for 'system wide' (2 runs):

S0-C0           2          722013519      cycles                                                      
S0-C0           2          420881666      instructions              #    0.58  insn per cycle         
S0-C0           2           10917452      cache-references                                            
S0-C0           2            3975059      cache-misses              #   36.410 % of all cache refs    
S0-C1           2          695895760      cycles                                                      
S0-C1           2          145568672      instructions              #    0.21  insn per cycle         
S0-C1           2            8308276      cache-references                                            
S0-C1           2            3231580      cache-misses              #   38.896 % of all cache refs    
S0-C2           2         1599077422      cycles                                                      
S0-C2           

#### Caso vectorizando

In [41]:
%%bash
sudo perf stat -S -e cycles,instructions,cache-references,cache-misses -r 2 python3 mult_matrix_vector_numpy.py


 Performance counter stats for 'python3 mult_matrix_vector_numpy.py' (2 runs):

      239426251422      cycles                                                        ( +-  0.20% )
      599033991518      instructions              #    2.50  insn per cycle           ( +-  0.09% )
         656790900      cache-references                                              ( +-  0.15% )
          93785326      cache-misses              #   14.279 % of all cache refs      ( +-  1.85% )

      59.527032712 seconds time elapsed                                          ( +-  0.25% )



In [42]:
%%bash
sudo perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 2 python3 mult_matrix_vector_numpy.py


 Performance counter stats for 'system wide' (2 runs):

S0-C0           2         1504209672      cycles                                                      
S0-C0           2         1066901784      instructions              #    0.71  insn per cycle         
S0-C0           2           12347468      cache-references                                            
S0-C0           2            3960360      cache-misses              #   32.074 % of all cache refs    
S0-C1           2         1935535192      cycles                                                      
S0-C1           2          538940580      instructions              #    0.28  insn per cycle         
S0-C1           2            6076215      cache-references                                            
S0-C1           2            2951868      cache-misses              #   48.581 % of all cache refs    
S0-C2           2         2116484334      cycles                                                      
S0-C2           

# Nivel 3 de BLAS

## Multiplicación de matrices

Para el "desempate" entre los tiempos de ejecución, *cache-references* y *cache-misses* entre la versión vectorizada y no vectorizada, consideremos al algoritmo de multiplicación de matrices $C = C + AB$ con $A \in \mathbb{R}^{m \times r}, B \in \mathbb{R}^{r \times n}, C \in \mathbb{R}^{m \times n}$. En LAPACK encontramos tal algoritmo con nombres como [sgemm](http://www.netlib.org/lapack/explore-html/d4/de2/sgemm_8f.html), [dgemm](http://www.netlib.org/lapack/explore-html/d7/d2b/dgemm_8f.html), [cgemm](http://www.netlib.org/lapack/explore-html/d6/d5b/cgemm_8f.html) y [zgemm](http://www.netlib.org/lapack/explore-html/d7/d76/zgemm_8f.html) para los casos de precisión simple, doble o números complejos respectivamente. Este algoritmo se cataloga como de nivel 3 de BLAS pues realiza una **cantidad de trabajo cuadrática** sobre una **cantidad cúbica de datos**. Ver [level 3](http://www.netlib.org/blas/#_level_3) para más ejemplos de algoritmos en el álgebra lineal en esta categoría.

El algoritmo de multiplicación de matrices puede escribirse en diferentes versiones. Por ejemplo la versión  **i,j,k** es:

```
for i in range(m):
    for j in range(n):
        for k in range(r):
            C[i][j]+=A[i][k]*B[k][j]
    
```

y la versión **j,k,i** es:

```
for j in range(n):
    for k in range(r):
        for i in range(m):
            C[i][j]+=A[i][k]*B[k][j]
```

**Comentarios:**

* Cualquiera de las versiones involucra una cantidad de trabajo del orden $\mathcal{O}(mnr)$, la cual es cúbica. Es posible interpretar ésta cantidad de trabajo con una frase del tipo "si duplicamos cada dimensión de $A$ entonces la cantidad de trabajo se incrementa por un factor de $8$".

* Distintas versiones tienen distinto patrón de acceso a los datos de $A, B$ y $C$. Por ejemplo, para la variante **i,j,k** el **inner loop** realiza un producto punto que requiere acceso a renglones de $A$ y columnas de $B$. La variante **j,k,i** involucra una operación *saxpy* en el **inner loop** y acceso por columnas a la matriz $C$ y a una columna de $A$. Un resúmen de lo anterior se presenta en el siguiente cuadro:

<img src="https://dl.dropboxusercontent.com/s/rw81crmo1b7fjvb/tabla_con_versiones_mult_matrices.png?dl=0" heigth="400" width="400">


### Comparación entre versiones vectorizadas y no vectorizadas

In [43]:
np.random.seed(2020)
m=10**4
r=10**4

A=np.random.rand(m,r)
fileA='A.txt'
np.savetxt(fileA,A)

In [44]:
np.random.seed(2021)
r=10**4
n=10**4

B=np.random.rand(r,n)
fileB='B.txt'
np.savetxt(fileB,B)

In [45]:
%%file mult_matrix_matrix.py
m=10**4
r=10**4
n=10**4

A = []
B = []
fileA='A.txt'
fileB='B.txt'

with open(fileA,'r') as f:
    for l in f:
        A.append([float(k) for k in l.replace('\n','').replace(' ',',').split(',')]) 
        
with open(fileB,'r') as f:
    for l in f:
        B.append([float(k) for k in l.replace('\n','').replace(' ',',').split(',')])  

C=[[0]*n for i in range(m)]

for i in range(m):
    for j in range(n):
        for k in range(r):
            C[i][j]+=A[i][k]*B[k][j]


Writing mult_matrix_matrix.py


In [46]:
%%file mult_matrix_matrix_numpy.py
import numpy as np
m=10**4
r=10**4
n=10**4

fileA='A.txt'
fileB='B.txt'
A = np.loadtxt(fileA)
B = np.loadtxt(fileB)
C = A@B

Writing mult_matrix_matrix_numpy.py


In [47]:
%%bash
sudo perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 1 python3 mult_matrix_matrix.py

Process is terminated.


... 36 min y todavía no terminaba... cambiar a r 1

In [1]:
%%bash
sudo perf stat -S -a -e cycles,instructions,cache-references,cache-misses -r 1 python3 mult_matrix_matrix.py

Process is terminated.


In [3]:
%%bash
sudo perf stat -S -a -e cycles,instructions,cache-references,cache-misses -r 2 python3 mult_matrix_matrix_numpy.py


 Performance counter stats for 'system wide' (2 runs):

      836044127523      cycles                                                        ( +-  0.57% )
     1686463020251      instructions              #    2.02  insn per cycle           ( +-  0.43% )
        4086628234      cache-references                                              ( +-  1.72% )
         882062956      cache-misses              #   21.584 % of all cache refs      ( +-  4.13% )

     130.609360500 seconds time elapsed                                          ( +-  0.66% )



In [2]:
%%bash
sudo perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 2 python3 mult_matrix_matrix_numpy.py


 Performance counter stats for 'system wide' (2 runs):

S0-C0           2        89616044167      cycles                                                      
S0-C0           2       121446222500      instructions              #    1.36  insn per cycle         
S0-C0           2          811435068      cache-references                                            
S0-C0           2          194368638      cache-misses              #   23.954 % of all cache refs    
S0-C1           2        88898510819      cycles                                                      
S0-C1           2       121389910135      instructions              #    1.37  insn per cycle         
S0-C1           2          722839867      cache-references                                            
S0-C1           2          162558289      cache-misses              #   22.489 % of all cache refs    
S0-C2           2       563600069053      cycles                                                      
S0-C2           

**Otra versión vectorizando el inner loop en el que se involucran productos punto:**

Pendiente:

In [193]:
%%file mult_matrix_matrix_numpy_dot_product.py
import numpy as np
m=10**2
r=10**3
n=10**2

fileA='A.txt'
fileB='B.txt'
A = np.loadtxt(fileA)
B = np.loadtxt(fileB)
C = np.zeros((m,n))
for i in np.arange(m):
        for j in np.arange(n):
                C[i][j]+= A[i,:].dot(B[:,j])

Overwriting mult_matrix_matrix_numpy_dot_product.py


In [194]:
%%bash
sudo perf stat -S -a -e cycles,instructions,cache-references,cache-misses -r 1 python3 mult_matrix_matrix_numpy_dot_product.py


 Performance counter stats for 'system wide' (5 runs):

        3052736247      cycles                                                        ( +-  2.05% )
        2680158414      instructions              #    0.88  insn per cycle           ( +-  0.72% )
          11041581      cache-references                                              ( +-  0.80% )
            644429      cache-misses              #    5.836 % of all cache refs      ( +-  7.31% )

       0.282211430 seconds time elapsed                                          ( +-  1.92% )



In [2]:
%%bash
sudo perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 1 python3 mult_matrix_matrix_numpy_dot_product.py


 Performance counter stats for 'system wide' (5 runs):

S0-C0           2          569610445      cycles                                                      
S0-C0           2          221505417      instructions              #    0.39  insn per cycle         
S0-C0           2              56811      cache-references                                            
S0-C0           2              17900      cache-misses              #   31.508 % of all cache refs    
S0-C1           2         1319889207      cycles                                                      
S0-C1           2         2002371463      instructions              #    1.52  insn per cycle         
S0-C1           2           10456781      cache-references                                            
S0-C1           2             489184      cache-misses              #    4.678 % of all cache refs    
S0-C2           2          570228997      cycles                                                      
S0-C2           

In [3]:
import os

In [4]:
os.remove('A.txt')
os.remove('B.txt')

**Referencias:**

* E. Anderson, Z. Bai, C. Bischof, L. S. Blackford, J. Demmel, J. Dongarra, J. Du Croz,
A. Greenbaum, S. Hammarling, A. Mckenney and D. Sorensen, LAPACK Users Guide, Society for Industrial and Applied Mathematics, Philadelphia, PA, third ed., 1999.

* G. H. Golub, C. F. Van Loan, Matrix Computations, John Hopkins University
Press, 2013.

* [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb)

Para más sobre BLAS, LAPACK con C ver:

* [C/BLAS](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/tree/master/C/BLAS)

* [C/LAPACK](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/tree/master/C/LAPACK)

Hay implementaciones en paralelo de BLAS para sistemas de memoria distribuida. Ver por ejemplo:

* [PBLAS](http://www.netlib.org/scalapack/pblas_qref.html) y [ScaLAPACK](http://www.netlib.org/scalapack/)

También NVIDIA tiene su propia implementación de BLAS para uso con GPU's: [CUBLAS](https://docs.nvidia.com/cuda/cublas/index.html) y su implementación de LAPACK: [CUSOLVER](https://docs.nvidia.com/cuda/cusolver/index.html). Para más sobre CUBLAS y CUSOLVER ver: [C/extensiones_a_C/CUDA/CUBLAS](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/tree/master/C/extensiones_a_C/CUDA/CUBLAS) y [C/extensiones_a_C/CUDA/CUSOLVER/](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/tree/master/C/extensiones_a_C/CUDA/CUSOLVER)

Otras referencias para uso de GPU's con implementaciones de BLAS y LAPACK se encuentran:

* [MAGMA](https://icl.cs.utk.edu/magma/), [MAGMA en NVIDIA](https://developer.nvidia.com/magma), ver por ejemplo: [Matrix computations on the GPU](https://developer.nvidia.com/sites/default/files/akamai/cuda/files/Misc/mygpu.pdf)

* [NVBLAS](https://docs.nvidia.com/cuda/nvblas/)

Otras referencias útiles para [perf](https://github.com/torvalds/linux/tree/master/tools/perf) son:

* [perf-wiki](https://perf.wiki.kernel.org/index.php/Tutorial)

* [perf-tools](https://github.com/brendangregg/perf-tools) para *tools* que se apoyan de `perf`.

